# Table of Contents

In [1]:
import os
import random
import time
import json
import warnings 
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler
from utils import label_accuracy_score, add_hist
import cv2

import numpy as np
import pandas as pd
from tqdm import tqdm

# 전처리를 위한 라이브러리
from pycocotools.coco import COCO
import torchvision
import torchvision.transforms as transforms

#!pip install albumentations==0.4.6
import albumentations as A
from albumentations.pytorch import ToTensorV2

# 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from matplotlib.patches import Patch

#!pip install webcolors
import webcolors

plt.rcParams['axes.grid'] = False

print('pytorch version: {}'.format(torch.__version__))
print('GPU 사용 가능 여부: {}'.format(torch.cuda.is_available()))

print(torch.cuda.get_device_name(0))
print(torch.cuda.device_count())

# GPU 사용 가능 여부에 따라 device 정보 저장
device = "cuda" if torch.cuda.is_available() else "cpu"

pytorch version: 1.13.1+cu117
GPU 사용 가능 여부: True
Tesla V100-PCIE-32GB
1


## 하이퍼파라미터 세팅 및 seed 고정

In [2]:
batch_size = 16   # Mini-batch size
num_epochs = 100
learning_rate = 0.0001

In [3]:
# seed 고정
random_seed = 42
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

## 학습 데이터 EDA

In [4]:
%matplotlib inline

dataset_path  = '/opt/ml/input/data'
anns_file_path = dataset_path + '/' + 'train_all.json'

# Read annotations
with open(anns_file_path, 'r') as f:
    dataset = json.loads(f.read())

categories = dataset['categories']
anns = dataset['annotations']
imgs = dataset['images']
nr_cats = len(categories)
nr_annotations = len(anns)
nr_images = len(imgs)

# Load categories and super categories
cat_names = []
super_cat_names = []
super_cat_ids = {}
super_cat_last_name = ''
nr_super_cats = 0
for cat_it in categories:
    cat_names.append(cat_it['name'])
    super_cat_name = cat_it['supercategory']
    # Adding new supercat
    if super_cat_name != super_cat_last_name:
        super_cat_names.append(super_cat_name)
        super_cat_ids[super_cat_name] = nr_super_cats
        super_cat_last_name = super_cat_name
        nr_super_cats += 1

print('Number of super categories:', nr_super_cats)
print('Number of categories:', nr_cats)
print('Number of annotations:', nr_annotations)
print('Number of images:', nr_images)

Number of super categories: 10
Number of categories: 10
Number of annotations: 26240
Number of images: 3272


In [5]:
# Count annotations
cat_histogram = np.zeros(nr_cats,dtype=int)
for ann in anns:
    cat_histogram[ann['category_id']-1] += 1

# Convert to DataFrame
df = pd.DataFrame({'Categories': cat_names, 'Number of annotations': cat_histogram})
df = df.sort_values('Number of annotations', 0, False)

In [6]:
# category labeling 
sorted_temp_df = df.sort_index()

# background = 0 에 해당되는 label 추가 후 기존들을 모두 label + 1 로 설정
sorted_df = pd.DataFrame(["Backgroud"], columns = ["Categories"])
sorted_df = sorted_df.append(sorted_temp_df, ignore_index=True)

In [7]:
# class (Categories) 에 따른 index 확인 (0~10 : 총 11개)
sorted_df

,Categories,Number of annotations
0,Backgroud,NaN
1,General trash,2782.0
2,Paper,9311.0
3,Paper pack,659.0
4,Metal,562.0
5,Glass,610.0
6,Plastic,3090.0
7,Styrofoam,1343.0
8,Plastic bag,7643.0
9,Battery,63.0


## COCO(data_dir) 데이터 전처리 함수 정의 (Dataset)

In [8]:
category_names = list(sorted_df.Categories)

def get_classname(classID, cats):
    for i in range(len(cats)):
        if cats[i]['id']==classID:
            return cats[i]['name']
    return "None"

class CustomDataLoader(Dataset):
    """COCO format"""
    def __init__(self, data_dir, mode = 'train', transform = None):
        super().__init__()
        self.mode = mode
        self.transform = transform
        self.coco = COCO(data_dir)
        
    def __getitem__(self, index: int):
        # dataset이 index되어 list처럼 동작
        image_id = self.coco.getImgIds(imgIds=index)
        image_infos = self.coco.loadImgs(image_id)[0]
        
        # cv2 를 활용하여 image 불러오기
        images = cv2.imread(os.path.join(dataset_path, image_infos['file_name']))
        images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB).astype(np.float32)
        images /= 255.0
        
        if (self.mode in ('train', 'val')):
            ann_ids = self.coco.getAnnIds(imgIds=image_infos['id'])
            anns = self.coco.loadAnns(ann_ids)

            # Load the categories in a variable
            cat_ids = self.coco.getCatIds()
            cats = self.coco.loadCats(cat_ids)

            # masks : size가 (height x width)인 2D
            # 각각의 pixel 값에는 "category id" 할당
            # Background = 0
            masks = np.zeros((image_infos["height"], image_infos["width"]))
            # General trash = 1, ... , Cigarette = 10
            anns = sorted(anns, key=lambda idx : idx['area'], reverse=True)
            for i in range(len(anns)):
                className = get_classname(anns[i]['category_id'], cats)
                pixel_value = category_names.index(className)
                masks[self.coco.annToMask(anns[i]) == 1] = pixel_value
            masks = masks.astype(np.int8)
                        
            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                transformed = self.transform(image=images, mask=masks)
                images = transformed["image"]
                masks = transformed["mask"]
            return images, masks, image_infos
        
        if self.mode == 'test':
            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                transformed = self.transform(image=images)
                images = transformed["image"]
            return images, image_infos
    
    def __len__(self) -> int:
        # 전체 dataset의 size를 return
        return len(self.coco.getImgIds())

## Dataset 정의 및 DataLoader 할당

In [9]:
# train.json / validation.json / test.json 디렉토리 설정
train_path = dataset_path + '/K-fold_train1.json'
val_path = dataset_path + '/K-fold_val1.json'
test_path = dataset_path + '/test.json'

# collate_fn needs for batch
def collate_fn(batch):
    return tuple(zip(*batch))


import albumentations as A
from albumentations.pytorch import ToTensorV2

train_transform = A.Compose([
                            #A.CropNonEmptyMaskIfExists(256,256,p=1),
                            A.HorizontalFlip(p=0.5),
                            A.OneOf([
                                A.ShiftScaleRotate(p=1),
                                A.RandomRotate90(p=1),
                            ], p=0.3),
                            A.OneOf([
                                A.Blur(blur_limit=3, p=1),
                                A.MotionBlur(blur_limit=3, p=1),
                                A.MedianBlur(blur_limit=3, p=1),
                                A.Sharpen(p=1),
                            ], p=0.2),
                            ToTensorV2()
                            ])
val_transform = A.Compose([
                          ToTensorV2()
                          ])

test_transform = A.Compose([
                           ToTensorV2()
                           ])

# create own Dataset
# train dataset
train_dataset = CustomDataLoader(data_dir=train_path, mode='train', transform=train_transform)

# validation dataset
val_dataset = CustomDataLoader(data_dir=val_path, mode='val', transform=val_transform)

# test dataset
test_dataset = CustomDataLoader(data_dir=test_path, mode='test', transform=test_transform)


# DataLoader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=4,
                                           collate_fn=collate_fn)

val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                         batch_size=batch_size,
                                         shuffle=False,
                                         num_workers=4,
                                         collate_fn=collate_fn)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          num_workers=4,
                                          collate_fn=collate_fn)

loading annotations into memory...
Done (t=10.14s)
creating index...
index created!
loading annotations into memory...
Done (t=2.28s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


## baseline model

### `smp.Unet()`

In [10]:
# jupyter command 에서 library download 하기
# !pip install git+https://github.com/qubvel/segmentation_models.pytorch

In [11]:
import segmentation_models_pytorch as smp

## train, validation, test 함수 정의

In [12]:
# wandb 시각화용 클래스 딕셔너리
class_labels = dict(sorted_df["Categories"])

In [13]:
import torch.cuda.amp as amp
scaler = amp.GradScaler(enabled=True)

In [14]:
import wandb

In [17]:
sweep_config = {
    'name' : 'HJ_smp_sweep', # sweep name
    'method': 'grid',        # grid, random, bayes
    'metric': {
        'name': 'Val/mIoU',
        'goal': 'maximize'   # 추적할 목표 
        },
    'parameters' : {
        'optimizer': {
                'values': ['AdamW', 'AdamP'] # 'Adam', 
        },
    }
}

In [18]:
import pprint

pprint.pprint(sweep_config)

{'method': 'grid',
 'metric': {'goal': 'maximize', 'name': 'Val/mIoU'},
 'name': 'HJ_smp_sweep',
 'parameters': {'optimizer': {'values': ['Adam', 'AdamW', 'AdamP']}}}


In [19]:
sweep_id = wandb.sweep(sweep_config, project="sweep_Unet++effv2") # 프로젝트명

Create sweep with ID: o8rvmjks
Sweep URL: https://wandb.ai/fullhouse/sweep_Unet%2B%2Beffv2/sweeps/o8rvmjks


## FocalTversky_loss 정의

In [20]:
def softmax_helper(x):
    # copy from: https://github.com/MIC-DKFZ/nnUNet/blob/master/nnunet/utilities/nd_softmax.py
    rpt = [1 for _ in range(len(x.size()))]
    rpt[1] = x.size(1)
    x_max = x.max(1, keepdim=True)[0].repeat(*rpt)
    e_x = torch.exp(x - x_max)
    return e_x / e_x.sum(1, keepdim=True).repeat(*rpt)

In [21]:
def sum_tensor(inp, axes, keepdim=False):
    # copy from: https://github.com/MIC-DKFZ/nnUNet/blob/master/nnunet/utilities/tensor_utilities.py
    axes = np.unique(axes).astype(int)
    if keepdim:
        for ax in axes:
            inp = inp.sum(int(ax), keepdim=True)
    else:
        for ax in sorted(axes, reverse=True):
            inp = inp.sum(int(ax))
    return inp

In [22]:
def get_tp_fp_fn(net_output, gt, axes=None, mask=None, square=False):
    """
    net_output must be (b, c, x, y(, z)))
    gt must be a label map (shape (b, 1, x, y(, z)) OR shape (b, x, y(, z))) or one hot encoding (b, c, x, y(, z))
    if mask is provided it must have shape (b, 1, x, y(, z)))
    :param net_output:
    :param gt:
    :param axes:
    :param mask: mask must be 1 for valid pixels and 0 for invalid pixels
    :param square: if True then fp, tp and fn will be squared before summation
    :return:
    """
    if axes is None:
        axes = tuple(range(2, len(net_output.size())))

    shp_x = net_output.shape
    shp_y = gt.shape

    with torch.no_grad():
        if len(shp_x) != len(shp_y):
            gt = gt.view((shp_y[0], 1, *shp_y[1:]))

        if all([i == j for i, j in zip(net_output.shape, gt.shape)]):
            # if this is the case then gt is probably already a one hot encoding
            y_onehot = gt
        else:
            gt = gt.long()
            y_onehot = torch.zeros(shp_x)
            if net_output.device.type == "cuda":
                y_onehot = y_onehot.cuda(net_output.device.index)
            y_onehot.scatter_(1, gt, 1)

    tp = net_output * y_onehot
    fp = net_output * (1 - y_onehot)
    fn = (1 - net_output) * y_onehot

    if mask is not None:
        tp = torch.stack(tuple(x_i * mask[:, 0] for x_i in torch.unbind(tp, dim=1)), dim=1)
        fp = torch.stack(tuple(x_i * mask[:, 0] for x_i in torch.unbind(fp, dim=1)), dim=1)
        fn = torch.stack(tuple(x_i * mask[:, 0] for x_i in torch.unbind(fn, dim=1)), dim=1)

    if square:
        tp = tp ** 2
        fp = fp ** 2
        fn = fn ** 2

    tp = sum_tensor(tp, axes, keepdim=False)
    fp = sum_tensor(fp, axes, keepdim=False)
    fn = sum_tensor(fn, axes, keepdim=False)

    return tp, fp, fn

In [23]:
class TverskyLoss(nn.Module):
    def __init__(self, apply_nonlin=None, batch_dice=False, do_bg=True, smooth=1.,
                 square=False):
        """
        paper: https://arxiv.org/pdf/1706.05721.pdf
        """
        super(TverskyLoss, self).__init__()

        self.square = square
        self.do_bg = do_bg
        self.batch_dice = batch_dice
        self.apply_nonlin = apply_nonlin
        self.smooth = smooth
        self.alpha = 0.3
        self.beta = 0.7

    def forward(self, x, y, loss_mask=None):
        shp_x = x.shape

        if self.batch_dice:
            axes = [0] + list(range(2, len(shp_x)))
        else:
            axes = list(range(2, len(shp_x)))

        if self.apply_nonlin is not None:
            x = self.apply_nonlin(x)

        tp, fp, fn = get_tp_fp_fn(x, y, axes, loss_mask, self.square)


        tversky = (tp + self.smooth) / (tp + self.alpha*fp + self.beta*fn + self.smooth)

        if not self.do_bg:
            if self.batch_dice:
                tversky = tversky[1:]
            else:
                tversky = tversky[:, 1:]
        tversky = tversky.mean()

        return -tversky

In [24]:
class FocalTversky_loss(nn.Module):
    """
    paper: https://arxiv.org/pdf/1810.07842.pdf
    author code: https://github.com/nabsabraham/focal-tversky-unet/blob/347d39117c24540400dfe80d106d2fb06d2b99e1/losses.py#L65
    """
    def __init__(self, tversky_kwargs, gamma=0.75):
        super(FocalTversky_loss, self).__init__()
        self.gamma = gamma
        self.tversky = TverskyLoss(**tversky_kwargs)

    def forward(self, net_output, target):
        tversky_loss = 1 + self.tversky(net_output, target) # = 1-tversky(net_output, target)
        focal_tversky = torch.pow(tversky_loss, self.gamma)
        return focal_tversky

In [25]:
# Loss function 정의
criterion = FocalTversky_loss({'apply_nonlin': softmax_helper, 'batch_dice': True, 'smooth': 1e-5, 'do_bg': True})
# early_stopping : 13번의 epoch 연속으로 val loss 미개선 시에 조기 종료
patience = 13

In [26]:
#!pip install git+https://github.com/ildoonet/pytorch-gradual-warmup-lr.git

In [27]:
# from warmup_scheduler import GradualWarmupScheduler

# scheduler_steplr = StepLR(optimizer, step_size=10, gamma=0.1)
# scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=1, total_epoch=5, after_scheduler=scheduler_steplr)

In [29]:
def train():
    model = smp.DeepLabV3Plus( #PlusPlus
        encoder_name="tu-tf_efficientnetv2_l", # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
        encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
        in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        classes=11,                     # model output channels (number of classes in your dataset)
        )
    
    
    default_config={
        "optimizer": 'Adam'}
    
    wandb.init(config=default_config)
    config = wandb.config
    
    if config.optimizer == 'Adam':
        optimizer = torch.optim.Adam(params = model.parameters(), lr = learning_rate, betas=(0.9, 0.999), weight_decay=1e-6)
    elif config.optimizer == 'AdamW':
        optimizer = torch.optim.AdamW(params = model.parameters(), lr = learning_rate, betas=(0.9, 0.999), weight_decay=1e-2)
    elif config.optimizer == 'AdamP':
        optimizer = AdamP(params=model.parameters(), lr=learning_rate, betas=(0.9, 0.999), weight_decay=1e-2)
        
    # scheduler 정의
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.3, patience=5, verbose=1)
    # scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max = 60, eta_min = 1e-5)
            
    print(f'Start training..')
    n_class = 11
    best_miou = 0

    for epoch in range(num_epochs):
        model.train()
        hist = np.zeros((n_class, n_class))
        for step, (images, masks, _) in enumerate(train_loader):
            images = torch.stack(images)       
            masks = torch.stack(masks).long() 
            # gpu 연산을 위해 device 할당
            images, masks = images.to(device), masks.to(device)

            # device 할당
            model = model.to(device)
            with amp.autocast(enabled=True):
                # inference
                outputs = model(images)

                # loss 계산 (cross entropy loss)
                loss = criterion(outputs, masks)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            # loss.backward()
            # optimizer.step()
            outputs = torch.argmax(outputs, dim=1).detach().cpu().numpy()
            masks = masks.detach().cpu().numpy()

            hist = add_hist(hist, masks, outputs, n_class=n_class)
            acc, acc_cls, mIoU, fwavacc, IoU = label_accuracy_score(hist)
            # step 주기에 따른 loss 출력
            if (step + 1) % 25 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{step+1}/{len(train_loader)}], \
                        Loss: {round(loss.item(),4)}, mIoU: {round(mIoU,4)}')
                wandb.log({"Train/Loss": round(loss.item(),4),
                           "Train/Accuracy": round(acc,4),
                           "Train/mIoU": round(mIoU,4)})
        wandb.log({"Charts/learning_rate": optimizer.param_groups[0]['lr']})
        
        #scheduler.step()
        
        # validation 주기에 따른 loss 출력 및 best model 저장
        if (epoch + 1) % 1 == 0:
            print(f'Start validation #{epoch +1}')
            val_miou = validation(model)
            
            scheduler.step(val_miou)
            
            if best_miou < val_miou:
                print('trigger times: 0')
                trigger_times = 0
                print(f"Best performance at epoch: {epoch + 1}")
                print(f"Save model in {saved_dir}")
                best_miou = val_miou
                save_model(model, saved_dir)
            else:
                trigger_times += 1
                print('Trigger Times:', trigger_times)
                if trigger_times >= patience:
                    print('Early stopping!\nStart to test process.')
                    return model

In [30]:
def validation(model):
    model.eval()

    with torch.no_grad():
        n_class = 11
        total_loss = 0
        cnt = 0
        
        hist = np.zeros((n_class, n_class))
        for step, (images, masks, _) in enumerate(val_loader):
            
            images = torch.stack(images)       
            masks = torch.stack(masks).long()  

            images, masks = images.to(device), masks.to(device)            
            
            # device 할당
            model = model.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, masks)
            total_loss += loss
            cnt += 1
            
            outputs = torch.argmax(outputs, dim=1).detach().cpu().numpy()
            masks = masks.detach().cpu().numpy()
            
            hist = add_hist(hist, masks, outputs, n_class=n_class)
             
            # step 주기에 따른 loss 출력
            if (step + 1) % 25 == 0:
                acc, acc_cls, mIoU, fwavacc, IoU = label_accuracy_score(hist)
                wandb.log({"Val/Loss": round(loss.item(),4),
                           "Val/Accuracy": round(acc,4),
                           "Val/mIoU": round(mIoU,4)})

        
        acc, acc_cls, mIoU, fwavacc, IoU = label_accuracy_score(hist)
        IoU_by_class = [{classes : round(IoU,4)} for IoU, classes in zip(IoU , sorted_df['Categories'])]
        
        avrg_loss = total_loss / cnt
        print(f'Validation Average Loss: {round(avrg_loss.item(), 4)}, Accuracy : {round(acc, 4)}, \
                mIoU: {round(mIoU, 4)}')
        print(f'IoU by class : {IoU_by_class}')
        
    return mIoU

In [32]:
# 모델 저장 함수 정의
val_every = 1

saved_dir = './saved'
# 저장 모델 이름 지정
model_names = 'effv2_unetpp_best.pt'
if not os.path.isdir(saved_dir):                                                           
    os.mkdir(saved_dir)

def save_model(model, saved_dir, file_name=model_names):
    output_path = os.path.join(saved_dir, file_name)
    torch.save(model.state_dict(), output_path)

In [33]:
# agent에서 위에서 정의한 train 함수를 15번 돌리는 과정
wandb.agent(sweep_id, train, count= 5)

wandb: Agent Starting Run: erhkygr3 with config:
wandb: 	optimizer: Adam
wandb: Currently logged in as: fullhouse (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade

CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.



Start training..
Epoch [1/100], Step [25/164],                         Loss: 0.9463, mIoU: 0.0444
Epoch [1/100], Step [50/164],                         Loss: 0.9219, mIoU: 0.0631
Epoch [1/100], Step [75/164],                         Loss: 0.8917, mIoU: 0.0816
Epoch [1/100], Step [100/164],                         Loss: 0.8712, mIoU: 0.1003
Epoch [1/100], Step [125/164],                         Loss: 0.8636, mIoU: 0.1155
Epoch [1/100], Step [150/164],                         Loss: 0.8368, mIoU: 0.1292
Start validation #1
Validation Average Loss: 0.8207, Accuracy : 0.8148,                 mIoU: 0.2468
IoU by class : [{'Backgroud': 0.8424}, {'General trash': 0.0004}, {'Paper': 0.4757}, {'Paper pack': 0.0001}, {'Metal': 0.0128}, {'Glass': 0.0}, {'Plastic': 0.2688}, {'Styrofoam': 0.4282}, {'Plastic bag': 0.6857}, {'Battery': 0.0}, {'Clothing': 0.0007}]
trigger times: 0
Best performance at epoch: 1
Save model in ./saved
Epoch [2/100], Step [25/164],                         Loss: 0.838, mIoU:

Charts/learning_rate,█████████████████████▃▃▃▃▃▃▃▃▃▃▃▃▃▁▁▁▁▁▁
Train/Accuracy,▁▆▇▇▇▇▇▇▇███████████████████████████████
Train/Loss,█▇▆▆▅▅▅▄▄▅▄▄▄▃▅▄▂▂▂▂▂▂▃▃▄▄▃▁▂▂▄▂▁▂▂▃▂▃▃▄
Train/mIoU,▁▃▃▄▄▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇██████████████████
Val/Accuracy,▁▅▆▆▇▆▇▇▇▇▇▇▇▇█▇▇▇█▇▇███████████████████
Val/Loss,█▆▅▅▄▄▃▃▂▁▃▅▄▃▃▂▃▁▂▄▃▃▃▃▂▄▃▃▃▃▃▃▃▂▃▂▂▂▂▂
Val/mIoU,▁▂▃▄▅▅▆▆▇▇▆▇▇▇▇▇▇▇▇▇▇██▇████████████████
Charts/learning_rate,0.0
Train/Accuracy,0.9619
Train/Loss,0.519
Train/mIoU,0.7087


wandb: Agent Starting Run: kz3qrmjg with config:
wandb: 	optimizer: AdamW
wandb: wandb version 0.13.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade

CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.



Start training..
Epoch [1/100], Step [25/164],                         Loss: 0.9603, mIoU: 0.0277
Epoch [1/100], Step [50/164],                         Loss: 0.9287, mIoU: 0.0485
Epoch [1/100], Step [75/164],                         Loss: 0.9017, mIoU: 0.0719
Epoch [1/100], Step [100/164],                         Loss: 0.8733, mIoU: 0.0913
Epoch [1/100], Step [125/164],                         Loss: 0.8512, mIoU: 0.1063
Epoch [1/100], Step [150/164],                         Loss: 0.8629, mIoU: 0.1185
Start validation #1
Validation Average Loss: 0.8431, Accuracy : 0.7965,                 mIoU: 0.205
IoU by class : [{'Backgroud': 0.8416}, {'General trash': 0.14}, {'Paper': 0.5067}, {'Paper pack': 0.0003}, {'Metal': 0.0}, {'Glass': 0.002}, {'Plastic': 0.0391}, {'Styrofoam': 0.002}, {'Plastic bag': 0.7231}, {'Battery': 0.0}, {'Clothing': 0.0001}]
trigger times: 0
Best performance at epoch: 1
Save model in ./saved
Epoch [2/100], Step [25/164],                         Loss: 0.823, mIoU: 0.20

Charts/learning_rate,███████████████████▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▁▁▁▁▁▁
Train/Accuracy,▁▆▇▇▇▇▇▇████████████████████████████████
Train/Loss,█▇▅▅▄▄▂▄▃▄▂▂▂▂▃▂▂▂▂▄▃▂▃▂▃▁▁▂▄▁▃▂▃▃▁▃▃▃▂▂
Train/mIoU,▁▂▃▄▄▅▆▆▆▆▆▇▇▇▇▇▇▇▇█████████████████████
Val/Accuracy,▁▄▆▆▆▆▇▇▇▇▇███▇█▇███████████████████████
Val/Loss,█▆▄▄▄▃▃▃▃▅▂▂▂▂▃▂▄▃▂▂▂▂▂▂▃▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂
Val/mIoU,▁▂▃▄▄▅▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇██▇▇███████████████
Charts/learning_rate,0.0
Train/Accuracy,0.9659
Train/Loss,0.4517
Train/mIoU,0.7827


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: cm7ajtq5 with config:
wandb: 	optimizer: AdamP
wandb: wandb version 0.13.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade

CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.



Start training..
Epoch [1/100], Step [25/164],                         Loss: 0.9639, mIoU: 0.0208
Epoch [1/100], Step [50/164],                         Loss: 0.9282, mIoU: 0.0358
Epoch [1/100], Step [75/164],                         Loss: 0.913, mIoU: 0.0581
Epoch [1/100], Step [100/164],                         Loss: 0.856, mIoU: 0.077
Epoch [1/100], Step [125/164],                         Loss: 0.8507, mIoU: 0.0937
Epoch [1/100], Step [150/164],                         Loss: 0.8705, mIoU: 0.1091
Start validation #1
Validation Average Loss: 0.8439, Accuracy : 0.7203,                 mIoU: 0.2383
IoU by class : [{'Backgroud': 0.7079}, {'General trash': 0.1588}, {'Paper': 0.5869}, {'Paper pack': 0.0071}, {'Metal': 0.0025}, {'Glass': 0.0063}, {'Plastic': 0.0154}, {'Styrofoam': 0.4458}, {'Plastic bag': 0.6896}, {'Battery': 0.0002}, {'Clothing': 0.0001}]
trigger times: 0
Best performance at epoch: 1
Save model in ./saved
Epoch [2/100], Step [25/164],                         Loss: 0.8281, m

Charts/learning_rate,█████████████▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▁▁▁▁▁
Train/Accuracy,▁▆▇▇▇███████████████████████████████████
Train/Loss,█▇▆▅▅▄▂▄▄▃▃▄▅▂▅▁▅▂▃▂▃▃▄▁▂▂▄▁▂▄▂▄▁▁▄▁▃▃▄▂
Train/mIoU,▁▃▄▄▅▆▆▆▇▇▇▇▇▇▇▇▇███████████████████████
Val/Accuracy,▁▅▆▇▇▇▇█████████████████████████████████
Val/Loss,█▆▅▄▃▂▃▃▂▂▃▁▂▃▂▂▂▂▂▃▂▃▂▂▂▃▂▂▂▁▃▂▂▂▂▂▂▂▂▂
Val/mIoU,▁▂▃▅▆▇▇▆▇▇▇▇▇▇▇▇▇███████████████▇███████
Charts/learning_rate,0.0
Train/Accuracy,0.9639
Train/Loss,0.4567
Train/mIoU,0.7158


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [ ]:
# best model 저장된 경로
model_path = os.path.join(saved_dir,model_names)

# best model 불러오기
model.load_state_dict(torch.load(model_path, map_location=device))

model = model.to(device)

In [ ]:
def test(model, data_loader, device):
    size = 256
    transform = A.Compose([A.Resize(size, size)])
    print('Start prediction.')
    
    model.eval()
    
    file_name_list = []
    preds_array = np.empty((0, size*size), dtype=np.long)
    
    with torch.no_grad():
        for step, (imgs, image_infos) in enumerate(tqdm(test_loader)):
            
            # inference (512 x 512)
            outs = model(torch.stack(imgs).to(device))
            oms = torch.argmax(outs.squeeze(), dim=1).detach().cpu().numpy()
            
            # resize (256 x 256)
            temp_mask = []
            for img, mask in zip(np.stack(imgs), oms):
                transformed = transform(image=img, mask=mask)
                mask = transformed['mask']
                temp_mask.append(mask)
                
            oms = np.array(temp_mask)
            
            oms = oms.reshape([oms.shape[0], size*size]).astype(int)
            preds_array = np.vstack((preds_array, oms))
            
            file_name_list.append([i['file_name'] for i in image_infos])
    print("End prediction.")
    file_names = [y for x in file_name_list for y in x]
    
    return file_names, preds_array

In [ ]:
# submission 파일이름
saved_csv = os.path.join('./submission',model_names[:-3]+".csv")
# sample_submisson.csv 열기
submission = pd.read_csv('./submission/sample_submission.csv', index_col=None)

# test set에 대한 prediction
file_names, preds = test(model, test_loader, device)

# PredictionString 대입
for file_name, string in zip(file_names, preds):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
submission.to_csv(saved_csv, index=False)